In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
from huggingface_hub import login
%matplotlib inline

In [13]:
df = pd.read_csv("analyzer_analyzer_urls.csv")
df_with_header = pd.concat([
    pd.DataFrame([df.columns], columns=df.columns),
    df
]).reset_index(drop=True)

# Переименовываем столбцы в простые индексы
df_with_header.columns = range(len(df.columns))
del df
df_with_header

,0,1
0,http://0-9.ru,"<html><head>\n <meta name=""viewport"" co..."
1,http://0009.ru,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
2,http://001k.ru,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
3,http://003ms.ru,"<!DOCTYPE html><html router=""application"" hexi..."
4,http://003rt.ru,"<html lang=""ru""><head>\n <meta name=""viewpo..."
...,...,...
9995,http://1hobby.ru,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
9996,http://1hod.ru,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
9997,http://1hop.ru,"<html class=""no-touch not-edge"" lang=""en-US""><..."
9998,http://1hostels.ru,"<html lang=""ru""><head>\n <title>Общежития и..."


In [23]:
print(f"Avg символов {df_with_header.iloc[:, 1].str.len().mean()}")

Avg символов 184867.6633


# CleanHTML.v1

In [ ]:
def clean_html(raw_html):
    """Удаление всей разметки и мусора"""
    import re
    from bs4 import BeautifulSoup
    

    soup = BeautifulSoup(raw_html, 'html.parser')

    # Удаление ненужных блоков
    for tag in ['script', 'style', 'meta', 'head', 'footer']:
        for el in soup.find_all(tag):
            el.decompose()

    # Извлечение ключевых секций
    main_content = soup.find(['main', 'article']) or soup.body
    text = main_content.get_text(separator=' ', strip=True) if main_content else ''

    # Удаление повторяющихся пробелов
    text = re.sub(r'\s+', ' ', text)

    #return text[:5000]  # Ограничение контекста для LLM
    return text          # Пока оставим так, т.к. подход не только LLMный

In [14]:
def clean_html(raw_html):
    """Удаление всей разметки и мусора"""
    import re
    from bs4 import BeautifulSoup
    

    soup = BeautifulSoup(raw_html, 'html.parser')

    # Удаление ненужных блоков
    for script in soup(['script', 'style', 'meta', 'head', 'footer']):
        script.decompose()

    # Извлечение ключевых секций
    #main_content = soup.find(['main', 'article']) or soup.body
    text = soup.get_text(separator='\n', strip=True)

    # Удаление повторяющихся пробелов
    text = re.sub(r'\s+', ' ', text)

    #return text[:5000]  # Ограничение контекста для LLM
    return text          # Пока оставим так, т.к. подход не только LLMный

In [15]:
import logging
from pandarallel import pandarallel
from tqdm.auto import tqdm


# Настраиваем логирование
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Инициализируем pandarallel с прогресс-баром
pandarallel.initialize(progress_bar=True)


second_column_name = df_with_header.columns[1]
logger.info(f"Обрабатываем столбец: {second_column_name}")

# Применяем функцию с прогресс-баром
tqdm.pandas(desc="Чистим HTML")
df_with_header[second_column_name] = df_with_header[second_column_name].parallel_apply(clean_html)

logger.info("Обработка завершена!")
#df_new = "analyzer_urls_without_marking.csv"
#df_with_header.to_csv(f"{df_new}")

#logger.info(f"Исходный датасет с обработкой сохранён под названием {df_new}")

2025-03-31 17:52:13,100 - INFO - Обрабатываем столбец: 1


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


2025-03-31 17:53:46,996 - INFO - Обработка завершена!


In [26]:
print(f"Avg символов {df_with_header.iloc[:, 1].str.len().mean()}")

Avg символов 6294.298


In [16]:
print(f"Avg символов {df_with_header.iloc[:, 1].str.len().mean()}")

Avg символов 7161.6295


In [ ]:
# Инициализация токенизатора
token = "" 
login(token=token)
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_llama3_8b")

# Функция для подсчёта токенов (оптимизированная версия)
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=True))

# Применяем ко всему датафрейму с прогресс-баром
tqdm.pandas(desc="Токенизация текстов")
df_with_header['token_count'] = df_with_header.iloc[:, 1].progress_apply(count_tokens)  # Второй столбец с текстами

# Рассчитываем среднее количество токенов
mean_tokens = df_with_header['token_count'].mean()
print(f"\nСреднее количество токенов на текст: {mean_tokens:.2f}")

# Дополнительно: статистика по количеству токенов
print("\nСтатистика по количеству токенов:")
print(df_with_header['token_count'].describe())

# Пример первых нескольких строк с токенами
print("\nПримеры первых 3 строк:")
for i, row in df_with_header.head(3).iterrows():
    text_sample = row[1][:50] + "..." if len(row[1]) > 50 else row[1]
    print(f"Строка {i}: {text_sample} -> {row['token_count']} токенов")

Токенизация текстов:   0%|          | 0/10000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (14238 > 8192). Running this sequence through the model will result in indexing errors



Среднее количество токенов на текст: 2200.66

Статистика по количеству токенов:
count     10000.000000
mean       2200.661200
std        8149.258486
min           1.000000
25%         477.750000
50%        1185.000000
75%        2555.250000
max      525868.000000
Name: token_count, dtype: float64

Примеры первых 3 строк:
Строка 0: Домен продается Рус Eng Купить в RU-CENTER Другие ... -> 24 токенов
Строка 1: Об Ассоциации Руководящие органы Из истории Ассоци... -> 203 токенов
Строка 2: Кирпичный завод №1. + 7 8442 53-07-07 Наше произво... -> 382 токенов


In [ ]:
# Инициализация токенизатора
token = "" 
login(token=token)
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_llama3_8b")

# Функция для подсчёта токенов (оптимизированная версия)
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=True))

# Применяем ко всему датафрейму с прогресс-баром
tqdm.pandas(desc="Токенизация текстов")
df_with_header['token_count'] = df_with_header.iloc[:, 1].progress_apply(count_tokens)  # Второй столбец с текстами

# Рассчитываем среднее количество токенов
mean_tokens = df_with_header['token_count'].mean()
print(f"\nСреднее количество токенов на текст: {mean_tokens:.2f}")

# Дополнительно: статистика по количеству токенов
print("\nСтатистика по количеству токенов:")
print(df_with_header['token_count'].describe())

# Пример первых нескольких строк с токенами
print("\nПримеры первых 3 строк:")
for i, row in df_with_header.head(3).iterrows():
    text_sample = row[1][:50] + "..." if len(row[1]) > 50 else row[1]
    print(f"Строка {i}: {text_sample} -> {row['token_count']} токенов")

Токенизация текстов:   0%|          | 0/10000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (14238 > 8192). Running this sequence through the model will result in indexing errors



Среднее количество токенов на текст: 2502.25

Статистика по количеству токенов:
count     10000.000000
mean       2502.251400
std        8186.524737
min           1.000000
25%         742.000000
50%        1488.000000
75%        2909.250000
max      525868.000000
Name: token_count, dtype: float64

Примеры первых 3 строк:
Строка 0: Домен продается Рус Eng Купить в RU-CENTER Другие ... -> 24 токенов
Строка 1: Об Ассоциации Руководящие органы Из истории Ассоци... -> 203 токенов
Строка 2: Перейти к содержанию Кирпичный завод №1 Меню Облиц... -> 430 токенов


In [18]:
df_with_header.iloc[2][1]

'Перейти к содержанию Кирпичный завод №1 Меню Облицовочный кирпич Контакты и схема проезда Прайс на облицовочный кирпич Облицовочный кирпич Кирпичный завод №1. + 7 8442 53-07-07 Наше производство Примеры работ Мы производим гиперпрессованный облицовочный кирпич по испанской технологии на современном оборудовании. Наше предприятие – это современное высокотехнологичное производство облицовочного кирпича методом гиперпрессования по инновационной испанской технологии. Новейшее оборудование завода позволяет выпускать продукцию, по всем характеристикам превышающую существующие Российские аналоги и соответствующую всем Российским и Европейским стандартам. Ассортимент выпускаемой продукции включает как широчайшую цветовую гамму, так и сортамент полнотелых, пустотелых, фасонных и «колотых» кирпичей. Завод производит одинарный и полуторный кирпич. Гиперпрессованный кирпич может применяться для кладки несущих конструкций зданий с сухим, нормальным и влажным режимом эксплуатации, в том числе наруж

In [19]:
df = pd.read_csv("analyzer_analyzer_urls.csv")
df = pd.concat([
    pd.DataFrame([df.columns], columns=df.columns),
    df
]).reset_index(drop=True)

# Переименовываем столбцы в простые индексы
df.columns = range(len(df.columns))

In [20]:
df.iloc[2][1]

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" lang="ru-RU" prefix="og: https://ogp.me/ns#"><head>\n<meta charset="UTF-8" />\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n<link rel="profile" href="http://gmpg.org/xfn/11" />\n<link rel="pingback" href="http://001k.ru/xmlrpc.php" />\n\n<title>Кирпичный завод №1 Облицовочный кирпич - Кирпичный завод №1</title>\n\n\t\t<!-- All in One SEO 4.4.1 - aioseo.com -->\n\t\t<meta name="description" content="Кирпичный завод №1 Кирпичный завод №1. + 7 8442 53-07-07Мы производим облицовочный гиперпрессованный кирпич по испанской технологии на современном оборудовании. Наше предприятие – это современное высокотехнологичное производство облицовочного кирпича методом гиперпрессования по инновационной испанской технологии. Новейшее оборудование завода позволяет выпускать продукцию, по всем характеристикам превышающую существующие Российские аналоги и соответствующую всем Российским и Европейским" />\n\t\t<m

In [ ]:
import pandas as pd
import spacy
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Загрузка модели для предобработки текста
nlp = spacy.load("ru_core_news_sm")

def enhanced_preprocessing(text):
    """Улучшенная предобработка текста с сохранением контактной информации"""
    doc = nlp(text)
    
    processed = []
    for token in doc:
        # Сохраняем числа и слова с цифрами (для телефонов)
        if token.like_num or any(c.isdigit() for c in token.text):
            processed.append(token.text)
            continue
            
        # Удаляем стоп-слова и пунктуацию, кроме специальных символов
        if not token.is_stop and not token.is_punct:
            # Лемматизация для остальных слов
            processed.append(token.lemma_.lower())
    
    return ' '.join(processed)

# Инициализация Gemma
model_name = "google/gemma-1.1-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

def extract_contacts(text):
    """Извлечение контактных данных с помощью Gemma"""
    prompt = f"""Проанализируй текст и извлеки контактную информацию. Ответь в формате JSON:
    {{
        "phones": [],
        "emails": [],
        "addresses": [],
        "social_media": [],
        "websites": []
    }}
    
    Текст для анализа: {text[:3000]}  # Ограничение контекста
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return parse_response(response)

def parse_response(response):
    """Парсинг ответа модели"""
    try:
        # Ищем JSON в ответе
        start = response.find('{')
        end = response.rfind('}') + 1
        return eval(response[start:end])
    except:
        return {
            "phones": [], 
            "emails": [], 
            "addresses": [],
            "social_media": [],
            "websites": []
        }

# Применение обработки
df_with_header['processed_text'] = df_with_header.iloc[:, 1].progress_apply(enhanced_preprocessing)
df_with_header['contacts'] = df_with_header['processed_text'].progress_apply(extract_contacts)

# Разбивка результатов на отдельные колонки
contact_cols = ['phones', 'emails', 'addresses', 'social_media', 'websites']
for col in contact_cols:
    df_with_header[col] = df_with_header['contacts'].apply(lambda x: x.get(col, []))